In [1]:
import pandas as pd # for data analytics
import numpy as np # for numerical computation
import pickle
from sklearn.metrics import precision_recall_fscore_support, classification_report,confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn import utils  
from sklearn import svm
from sklearn.model_selection import train_test_split
from fastai.structured import *
# from fastai.column_data import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_pickle('/home/chance/Desktop/hot_article_df.pickle').rename(columns={'over_10w':'Class'})
df.loc[df['Class'] == 1, "Class"] = -1
df.loc[df['Class'] == 0, "Class"] = 1

In [3]:
# pos = df[df['Class']==1]
# neg = df[df['Class']==-1]
# pos.shape,neg.shape,pos.shape[0]/df.shape[0]

In [4]:
# df = pd.concat([pos.head(30000),neg])
# df['Class'].value_counts()

In [5]:
#getting random set of nonfraud data to train on
non_fraud = df[df['Class']==1]
df_train, val = train_test_split(non_fraud, test_size=0.20, random_state=42)
fraud = df[df['Class']==-1]

In [6]:
proc_df(df_train, 'Class', do_scale=True)[3]

DataFrameMapper(default=False, df_out=False,
        features=[(['title_emotion'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['title_car_listed'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['title_car_price'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['title_car_apperance'], StandardScaler(copy=Tr...h_mean=True, with_std=True)), (['pos_v'], StandardScaler(copy=True, with_mean=True, with_std=True))],
        input_df=False, sparse=False)

In [7]:
#fastai
df, _, nas, mapper = proc_df(df_train, 'Class', do_scale=True)
df_val, _, nas, mapper = proc_df(val, 'Class', mapper=mapper, na_dict=nas, do_scale=True)
df_fraud, _, nas, mapper = proc_df(fraud, 'Class', mapper=mapper, na_dict=nas, do_scale=True)

In [8]:
model = svm.OneClassSVM(kernel='rbf', nu=0.0005,gamma=0.007)
model.fit(df)
model

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.007, kernel='rbf',
      max_iter=-1, nu=0.0005, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [9]:
#Creating a test set that contains both fraud and non fraud
y_val = val['Class']
y_fraud = fraud['Class']
y_testval = pd.concat([y_val, y_fraud])
y_testval = np.array(y_testval)
df_testval = pd.concat([df_val, df_fraud])

In [10]:
#predicting on test set, which consists of both fraud and non-fraud
pred_testval = model.predict(df_testval)
pred_testval,len(pred_testval)

(array([1, 1, 1, ..., 1, 1, 1]), 11516)

In [11]:
print(classification_report(y_testval, pred_testval))

              precision    recall  f1-score   support

          -1       0.10      0.00      0.00       947
           1       0.92      1.00      0.96     10569

   micro avg       0.92      0.92      0.92     11516
   macro avg       0.51      0.50      0.48     11516
weighted avg       0.85      0.92      0.88     11516



In [12]:
prec, rec, f2, _ = precision_recall_fscore_support(y_testval, pred_testval, beta=2, 
                                                   pos_label=-1, average='binary')
print(f'precision is {prec}, recall is {rec} and F2 score is {f2}')

precision is 0.1, recall is 0.0010559662090813093 and F2 score is 0.0013164823591363874


In [13]:
roc = roc_auc_score(y_testval, pred_testval)
print(f'ROC score is {roc}')

ROC score is 0.5001022096160366
